In [1]:
# %pip install -U transformers

In [2]:
# %pip install -U transformers[torch]

In [3]:
# %pip install -U accelerate 

In [4]:
import accelerate
print(accelerate.__version__)

1.11.0


d:\Project-to-learn\.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# %pip install datasets

In [6]:
# %pip install evaluate

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np
import torch

In [8]:
datasets=load_dataset("imdb")


In [9]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [11]:
def tokenizer_function(examples):
    return tokenizer(examples['text'],padding='max_length', truncation=True)


tokenized_datasets = datasets.map(tokenizer_function, batched=True)

In [12]:
tokenized_datasets["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [13]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# metric = evaluate.load("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [15]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)


In [17]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(2000)),  # small subset for speed
    eval_dataset=tokenized_datasets["test"].select(range(1000)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


C:\Users\sande\AppData\Local\Temp\ipykernel_19472\2257448161.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
trainer.train()

Step,Training Loss
50,0.653900
100,0.342100
150,0.363000
200,0.337900
250,0.354200
300,0.220800
350,0.224900
400,0.153300
450,0.211500
500,0.144500


TrainOutput(global_step=500, training_loss=0.3006249132156372, metrics={'train_runtime': 291.1757, 'train_samples_per_second': 13.737, 'train_steps_per_second': 1.717, 'total_flos': 1052444221440000.0, 'train_loss': 0.3006249132156372, 'epoch': 2.0})

In [20]:
trainer.evaluate()


{'eval_loss': 0.3176559805870056,
 'eval_accuracy': 0.916,
 'eval_runtime': 27.5724,
 'eval_samples_per_second': 36.268,
 'eval_steps_per_second': 4.534,
 'epoch': 2.0}

In [22]:
text = [
    "This movie was absolutely amazing!",
    "I didn’t like the film at all. It was boring and slow."
]

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
trainer.save_model("./bert-imdb-model")
tokenizer.save_pretrained("./bert-imdb-model")


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("./bert-imdb-model")
tokenizer = AutoTokenizer.from_pretrained("./bert-imdb-model")
